In [1]:
import os

import json
import requests as r

from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

In [2]:
embedding = HuggingFaceEmbeddings(model_name='maidalun1020/bce-embedding-base_v1')

db_persistent_dir = '../blog-vecdb'

if os.path.exists(db_persistent_dir):
    db = FAISS.load_local(db_persistent_dir, embedding, allow_dangerous_deserialization=True)
    print('成功从 {} 中提取数据'.format(db_persistent_dir))
else:
    loader = DirectoryLoader('../docs', glob='**/*.md')
    docs = loader.load()
    print('整理得到 {} 个文档'.format(len(docs)))

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 800,
        chunk_overlap  = 80,
        length_function = len,
        add_start_index = True
    )
    split_docs = text_splitter.split_documents(docs)
    print('分块得到 {} 个文档'.format(len(split_docs)))
    db = FAISS.from_documents(split_docs, embedding)
    db.save_local(db_persistent_dir)
    print('数据库已存储到 {} 中'.format(db_persistent_dir))

/data/zhelonghuang/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/zhelonghuang/miniconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


整理得到 304 个文档
分块得到 1276 个文档
数据库已存储到 ../blog-vecdb 中


In [3]:
db.similarity_search_with_score(
    query='vcd 波形文件的基本格式是什么样子的？',
    k=3
)

[(Document(page_content='完整的 VCD 语法，可以参考中科大资源网站上的 IEEE 1364 标准 第 18 章的内容。\n\n基本信息\n\n一个 vcd 文件会描述波形产生的基本信息，包括，日期，版本，单位时间，注释等等。\n\n日期 date\n\n$date\n    Sat Apr 20 20:06:14 2024\n$end\n\n与 verilog 类似，$end$ 是一切 scope 的结束符号，你也可以写成这样：\n\n$date Sat Apr 20 20:06:14 2024 $end\n\n版本 version\n\n$version\n    Icarus Verilog\n$end\n\n时间单位 timescale\n\n$timescale\n    1ns\n$end\n\n注释 comment\n\n$comment\n    Show the parameter values.\n$end\n\n变量申明\n\nvcd 中通过如下的方式申明一个 module 内的变量，，每条的格式为 $var type bitwidth id name，比如：\n\n$scope module ID_EX $end\n$var wire 2 ! AluOp [1:0] $end\n$var wire 1 " AluSrc $end\n$var wire 1 # MemRead $end\n$upscope $end\n\n以上语句申明了一个模块 ID_EX 和内部的三个信号 AluOp，AluSrc 和 MemRead。这三个变量在接下来的 vcd 描述中会被 !，"，# 替代，这也被称为 vcd 描述中，变量的 id.\n\n单个 module 的变量申明通过 $upscope $end 结束。\n\n所有 module 的变量申明通过 $enddefinitions $end 结束。\n\nparameter 赋值', metadata={'source': '../docs/kirigaya.cn/224.md', 'start_index': 0}),
  0.4351002),
 (Document(page_content='TODO\n\n找到所有 vcd item 的 type 类型，可参考：https

In [4]:
db.similarity_search_with_score(
    query='digital ide 有什么教程吗',
    k=3
)

[(Document(page_content='问题标题 基础教程太少\n除了b站的两个教程外，建议多出文档以及示例工程，一个刚接触FPGA但有不熟悉vivado的萌新就指望这个加快学习进度了\n请问B站教程标题是啥？我搜digital-ide啥也搜不到\n请问B站教程标题是啥？我搜digital-ide啥也搜不到\n《Digital-IDE使用教程(一个DDS的实现)》\n《Digital-IDE应用之FM调制解调》\n请问B站教程标题是啥？我搜digital-ide啥也搜不到\n《Digital-IDE使用教程(一个DDS的实现)》 《Digital-IDE应用之FM调制解调》\n好的谢谢\n那个教程是好早之前我出的了，新的版本一直不稳定，今年稳定之后会出后续教程。\nDigital IDE 的使用教程可以看\n官方文档为 https://sterben.nitcloud.cn/zh/ 但是目前文档不是很完善。', metadata={'source': '../docs/digital-issue/issue-47/issue.md', 'start_index': 0}),
  0.63593847),
 (Document(page_content='home: true\nheroImage: /icon.png\ndescription: Vscode 平台上的 ASIC & FPGA 开发扩展\nactionText: 快速开始 🐳\nactionLink: /zh/guide/introduction\nfeatures:\n- title: ✨ HDL 语言支持\n  details: 支持 verilog, vhdl, systemverilog, tcl 脚本等\n- title: 🎯 项目管理\n  details: 在你的项目中查看结构化的 HDL 文件\n- title: 🛠️ 额外的工具\n  details: FSM, Netlist, 一键仿真, 文档化，让你的编程体验更加舒坦。\n\n::: slot footer\nMIT Licensed | Copyright © 2018-present Digital-EDA\n:::', metadata={'source': '../docs/digital-document/i

In [7]:
import os
import json
import requests as r


api_key = '9J5qFTYr6wPRxkoVoXycnoWf'
secret_key = 'Xa6eJelStx5i7Ft3qQH0NAT6AvOkqhkH'

def get_access_key():
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    url = f'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={api_key}&client_secret={secret_key}'
    payload = json.dumps("")

    res = r.post(
        url=url,
        data=payload,
        headers=headers
    )

    resJson = res.json()
    access_token = resJson.get('access_token')
    assert isinstance(access_token, str), 'access_token 获取失败，详细信息' + str(resJson)
    return access_token

access_token = get_access_key()

In [35]:
def make_prompts(docs: list[str]) -> str:
    prompts = '你是一个很聪明的AI，下面是你的知识库和参考内容，请根据参考内容回答上述的问题：\n{}'.format('\n'.join(docs))
    return prompts

def call_llm(query: str) -> str:
    payload = json.dumps({
        "messages": [
            { "role": "user", "content": query }
        ]
    })

    headers = {
        'Content-Type': 'application/json'
    }
    try:
        url = 'https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/ernie-lite-8k?access_token=' + access_token
        res = r.post(url, headers=headers, data=payload)
        return res.json()['result']
    except Exception as e:
        return '发生错误：' + e.__str__()

def make_query(query, use_vecdb=True, threshold=0.6):
    if not use_vecdb:
        return call_llm(query)
    
    doc_scores = db.similarity_search_with_score(query, k=3)
    print(doc_scores)
    suggest_docs = []
    for doc, score in doc_scores:
        if score < threshold:
            detail = '详细请参考： https://kirigaya.cn/blog/article?seq=232'
            page_content = doc.page_content.strip() + '\n' + detail.strip()
            suggest_docs.append(page_content)
    
    if len(suggest_docs) > 0:
        prompt = make_prompts(suggest_docs)
        question = query + '\n' + prompt
        answer = call_llm(question)
        return answer
    else:
        return '我不知道'

print(make_query('property.json'))

[(Document(page_content='对于property.json文件中的library属性不建议用户自己配置，建议使用import library命令，或者下图中的图标激活命令进行配置。\n\nstate 属性代表是库文件是加载到本地工作区，还是作为远程进行链接。\n- remote 代表从远程虚拟包含（不在工作区下的都被认为远程，而不是网络上的远程）。\n  - 远程库文件可以打开并更改 (注：如果更改之后下次导入就是更改之后的代码) 。\n- local  代表将远程文件导入到该工程本地\n  1. 放置到arch.hardware.src下的lib中，此时更改不会影响远程库中的代码。\n  2. 注：当从local改回remote时lib文件夹会被删除（插件会提醒），请注意。\n\ncommon 属性代表插件自带的HDL功能库，该库的代码不太成熟，仅供参考。\n目前已经经过仿真测试的lib路径如下\n- Soc\n- Math/Cordic.v\n- Math/Sort3.v\n- Math/Sqrt.v\n- Malloc/RAM/Shift_RAM\n- Apply/DSP/Advance/Communicate/Modulate\n- Apply/DSP/Base/DDS\n- Apply/Image  (需要包含 Sort3, Sqrt, Shift_RAM)\n\n【注】：当输入的是文件夹时则包含该文件夹下所有的文件。此外不建议直接更改该库中的代码，更改之后再在下一次插件更新之后会被重新覆盖，请慎重。\n\ncustom 属性代表用户自定义HDL功能库。\n该属性的使用需要对setting下的PRJ.customer.Lib.repo.path进行配置用户自定义库的根目录，并与custom属性下的配置组成文件(夹)的绝对路径。表示如下：\nPRJ.customer.Lib.repo.path/${custom}', metadata={'source': 'docs/digital-document/guide/pm-project-building.md', 'start_index': 2533}), 0.6334237), (Document(page_content='json\n{\n    "version": "0.2.

In [34]:
print(make_query('给我 jdk 的下载链接', threshold=0.1))

[(Document(page_content='我的服务器（openjdk "11.0.20"）是满足的。\n\n设置环境变量 JAVA_HOME\n\n检查你是否有环境变量JAVA_HOME：\nbash\n$ echo $JAVA_HOME\n/usr/lib/jvm/java-11-openjdk-amd64\n我也是有的，没有的话，记得在~/.bashrc或者~/.zshrc中export一下。\n\nglibc 版本 > 2.17\n\n然后检查你的glibc版本，保证glibc至少支持到2.17的版本：\n\nbash\n$ strings /lib64/libc.so.6 | grep GLIBC_\nGLIBC_2.2.5\n...\n...\nGLIBC_2.30\nGLIBC_PRIVATE\n\n如果不满足的话，记得升级一下glibc，如何升级，请看这篇博客。\n\n因为下面要启动两个持久化服务，如果你没有后台运行软件，可以用用screen这个工具：\nbash\n$ sudo apt install screen\n\n安装签名服务器\n\n在安装Mirai之前，一定要先安装好签名服务器，并运行，否则我们将无法正常进行登录。\n\n下载项目 unidbg-fetch-qsign 的release\n\n在unidbg-fetch-qsign项目的 release 中下载最新版本，比如我用的版本是这样的：\n\nbash\n$ mkdir -p ~/Package/qsign && cd ~/Package/qsign\n$ wget https://github.com/fuqiuluo/unidbg-fetch-qsign/releases/download/1.1.9/unidbg-fetch-qsign.3.zip\n\n连续unzip两次得到文件夹，文件夹内部一共三个文件夹：bin, lib, txlib。进入txlib查看支持协议版本：', metadata={'source': 'docs/kirigaya.cn/104.md', 'start_index': 654}), 0.7496957), (Document(page_content='前言\n\n登录失效\n\n最近因为要使用QQ机器人来给群友回答一些技术问题，所以在

In [18]:
print(make_query('如何制作核武器？', use_vecdb=True))

我不知道


In [19]:
print(make_query('如何制作核武器？', use_vecdb=False))

制作核武器是一个高度敏感且极端复杂的科学问题，需要专业的知识和技术。同时，核武器是一种极具破坏性和毁灭性的武器，其制造和使用将导致不可估量的后果和灾难。因此，我无法提供任何关于如何制作核武器的信息或建议。
核武器的制造和使用是违反国际法和道德准则的，会带来严重的后果和灾难性影响。我们应该尊重国际法和道德准则，维护世界和平与稳定。如果您对核武器或其他相关问题感兴趣，建议您咨询专业人士或查阅相关文献资料，以获取更准确和全面的信息。


In [22]:
print(make_query('vcd 波形文件的格式是什么样的？', use_vecdb=False))

VCD（Visual Coded Data）波形文件是一种用于描述电子电路中信号波形的文件格式。VCD波形文件通常以文本文件的形式存储，其中包含了信号的采样数据和时间信息。

VCD波形文件的格式并没有一个严格的标准，不同的软件或工具可能会采用不同的格式来存储波形数据。然而，通常VCD波形文件包含以下基本元素：

1. 头信息（Header）：这部分包含了文件的元数据，如文件名、文件版本、时间单位等。
2. 信号定义（Signal Definitions）：这部分定义了波形文件中包含的信号名称、颜色、刻度等属性。
3. 采样数据（Sample Data）：这是波形文件的核心部分，包含了信号的采样数据和时间信息。通常以二维数组的形式存储，其中每个点代表一个采样点的幅度和时间。

VCD波形文件的具体格式可能会因不同的软件或工具而有所不同，但一般来说，它们都是文本可读的格式，可以方便地使用文本编辑器或其他工具进行查看和编辑。此外，一些电子设计自动化（EDA）工具也支持导入和导出VCD波形文件，以便于进行电路仿真和调试等操作。

需要注意的是，除了VCD格式外，还有其他一些用于描述波形数据的文件格式，如ASCII、CSV、HDL-TNS等。每种格式都有其特点和适用范围，具体选择哪种格式取决于实际需求和使用的工具。
